In [1]:
using BandedMatrices



In [2]:
type parameters
    nX :: Int # Number of spatial mesh points
    nT :: Int # Number of timesteps
    nU :: Int # Number of state variables per time step
    nZ :: Int # Number of control variables per time step
    theta :: Float64 # time-stepping parameter
    gamma :: Float64 # Viscosity Parameter
    omega :: Float64 # Control Penalty Parameter
    
    dx :: Float64 # Difference between any two mesh nodes, (= h)
    x :: Array{Float64,1} # Mesh nodes. 
    dt :: Float64 # Difference between any two time steps
    t :: Array{Float64,1} # Time steps 
    
    #u0 :: Array{Float64,1} # State at time 0. Coded as function
    #z0 :: Array{Float64,2} # Initial guess for Control
    #w :: Array{Float64,2} # Desired state. Coded as function
    r :: Array{Float64,2} # Problem Data.
    g :: Array{Float64,2} # Problem Data
    
    M :: SymTridiagonal{Float64} # Mass Matrix
    A :: SymTridiagonal{Float64} # Stiffness Matrix 
    B :: BandedMatrix{Float64}  
    Q :: SymTridiagonal{Float64}  
    
    G :: SymTridiagonal{Float64} 
    H :: SymTridiagonal{Float64}
    
    function parameters(nX = 80, nT = 80, theta = 0.5, gamma = 0.05, omega = 1e-2)
        nU = nX - 1
        nZ = nX + 1
        dx = 1/nX
        x = Array(linspace(0,1,nX))
        dt = 1/nT
        t = Array(linspace(0,1,nT))
        
        r = zeros(nX-1,nT+1)
        g = zeros(nX-1,nT+1)
        nX2 = Int(floor(nX/2))
        g[1:nX2,:] = -dx*ones(nX2,nT+1)
        
        M,A,B,Q = buildFEM(nX,gamma)
    
        G = theta*dt*A + M
        H = theta*dt*A - M
        
        new(nX,nT,nU,nZ,theta,gamma,omega,dx,x,dt,t,r,g,M,A,B,Q,G,H)
    end 
end 

function w(x::Float64, t::Float64)
    if(x<=1/2) return 1 end
    return 0
end 

#function r(x::Float64, t::Float64)
#    return 0
#end 

function u0(x::Float64)
    return w(x,0)
end

function Nu(u::Array{Float64,1})
    len = length(u)
    val = zeros(size(u))
    
    val[1] = u[1]*u[2] + u[2]^2
    
    for i in 2:len-1
        val[i] = -u[i-1]^2 + - u[i-1]*u[i] + u[i]*u[i+1]+u[i+1]^2
    end
    
    val[len] = -u[len-1]^2 - u[len-1]*u[len]
    
    val /= 6
    
    return val        
end

function Nu_deriv(u::Array{Float64,1})
    len = length(u)
    
    dl = -2*u[1:len-1] - u[2:len]
    
    d = zeros(len)
    d[1] = u[2]
    d[2:len-1] = u[3:len] - u[1:len-2]
    d[len] = -u[len-1]
    
    du = u[1:len-1] + 2*u[2:len] 
    
    val = Tridiagonal(dl,d,du)
    
    return val
end 


Nu_deriv (generic function with 1 method)

In [3]:
function buildFEM(n::Int, gamma::Float64)
    h = 1/n

    o = h/6*ones(n-2)
    d = 4*h/6*ones(n-1)
    M = SymTridiagonal(d,o)
    
    o = (-gamma/h)*ones(n-2)
    d = ((2*gamma)/h)*ones(n-1)
    A = SymTridiagonal(d,o)
    
    B = bones(Float64,n-1,n+1,0,2)
    for i in 1:n-1
        B[i,i+1] = 4
    end 
    B = (-h/6)*B
    
    o = h/6*ones(n)
    d = 4*h/6*ones(n+1)
    d[1]   = 2*h/6
    d[n+1] = 2*h/6
    Q = SymTridiagonal(d,o)
    
    return M,A,B,Q
end 

buildFEM (generic function with 1 method)

In [12]:
function solve_state!(Z::Array{Float64,2}, U::Array{Float64,2}, p::parameters)
    nT = p.nT
    nX = p.nX
    dt = p.dt
    dx = p.dx
    theta = p.theta
    B = p.B
    G = p.G
    H = p.H
    r = p.r 
    
    # time step i
    iflag = 0
    idebg = 0 
    
    # compute y(:,i+1) using Newton's method
    tol      = 1.e-2*min(dt^2,dx^2)
    max_iter = 10
    
    # initial time
    rhs = zeros(nX-1)
    nX2 = Int(floor(nX/2))
    rhs[1:nX2] = ones(nX2)
    U[:,1] = rhs
    
    for i = 1:nT
        # initial guess 
        U[:,i+1] = U[:,i]
      
        # Residual (res0 is the residual component that is independent of y(:,i+1))
        res0 = H*U[:,i] + theta*dt*Nu(U[:,i]) + theta*dt*(B*(Z[:,i] + Z[:,i+1])) - theta*dt*(r[:,i] + r[:,i+1])
        res  = res0 + G*U[:,i+1] + theta*dt*Nu(U[:,i+1])
    
        iter = 0
    
        if( idebg > 0 )
            println("   time step    Newton iter   Residual    Step size")
        end
      
        while (iter < max_iter && norm(res) >= tol )
      
            resnorm = norm(res);
            if( idebg > 0 )
                if( iter == 0 )
                    println([ i    iter     resnorm ])
                else
                    println([ i    iter     resnorm  stepsize ])
                end
            end
           
        
            # Jacobian
            Mat = G + theta*dt*Nu_deriv(U[:,i+1])
           
            # Newton step
            stepU = -Mat\res
           
            # Compue new guess using Armijo rule
            stepsize = 1         # step size
            U_tmp     = U[:,i+1] + stepsize*stepU
            res      = res0 + G*U_tmp + theta*dt*Nu(U_tmp)
           
            while ( norm(res) >= (1-1.e-4*stepsize)*resnorm )
                stepsize = stepsize/2
                U_tmp     = U[:,i+1] + stepsize*stepU
                res      = res0 + G*U_tmp + theta*dt*Nu(U_tmp)
            end
            U[:,i+1] = U_tmp
           
            iter = iter+1
        end
    end
end 

solve_state! (generic function with 1 method)

In [5]:
function evaluate_objective(Z::Array{Float64,2}, U::Array{Float64,2}, p::parameters)
    dt = p.dt   # length of time interval
    dx = p.dx   # length of spatial interval
    nT = p.nT       # number of time intervals
    nX = p.nX       # number of spatial intervals
    omega = p.omega    # conTrol penalty
    M = p.M
    g = p.g
    Q = p.Q
    
    val = (1/2)*(U[:,1]'*M*U[:,1]) + U[:,1]'*g[:,1] + (omega/2)*(Z[:,1]'*Q*Z[:,1])
    
    for i = 2:nT
        val += U[:,i]'*M*U[:,i] + 2*(U[:,i]'*g[:,i]) + omega*(Z[:,i]'*Q*Z[:,i])
    end
    
    val += (1/2)*(U[:,nT+1]'*M*U[:,nT+1]) + U[:,nT+1]'*g[:,nT+1] + (omega/2)*(Z[:,nT+1]'*Q*Z[:,nT+1])

    val = 0.5*dt*val
    
    return val[1]
end 

evaluate_objective (generic function with 1 method)

In [14]:
function evaluate_gradient!(Z::Array{Float64,2}, U::Array{Float64,2}, grad::Array{Float64,2}, p::parameters)
    nT = p.nT
    nX = p.nX
    nU = p.nU
    nZ = p.nZ
    dt = p.dt
    omega = p.omega
    theta = p.theta
    
    M = p.M
    Q = p.Q
    G = p.G
    H = p.H
    g = p.g
    Bt   = p.B'
    
    L2 = zeros(nU) # adjoint at i+1
    L1 = zeros(nU) # adjoint at i
    
    Mat = G + theta*dt*Nu_deriv(U[:,nT+1])
    rhs = -theta*dt*( M*U[:,nT+1] + g[:,nT+1] )
    L2 = Mat'\rhs
    
    grad[:,nT+1] = theta * dt* ( omega*Q*Z[:,nT+1] + Bt*L2 )

    
    for i in nT:-1:2
        Mat = G + theta*dt*Nu_deriv(U[:,i])
        rhs = -( H + theta*dt*Nu_deriv(U[:,i]) )'*L2 - dt*( M*U[:,i] + g[:,i] )
        L1  = Mat'\rhs
        
        grad[:,i] = dt * ( omega*Q*Z[:,i] + theta*Bt*(L1 + L2) )
        
        L2 = L1
    end
    
    grad[:,1] = theta*dt*( omega*Q*Z[:,1] + Bt*L2)
end 

evaluate_gradient! (generic function with 1 method)

In [38]:
function minimize(z0::Array{Float64,2}, p::parameters)
    step_size = 1.0
    tol = 1e-3
    
    z = z0
    val = 0.0
    U = zeros(p.nU,p.nT+1)
    grad = zeros(p.nZ,p.nT+1)
    
    α = 1 # α ∈ (0,0.5)
    β = 0.9  # β ∈ (0,1)

    
    values = []
    max_iter = 1000
    
    for i in 1:max_iter
        # solve state equation at current z
        #println("Solving State")
        solve_state!(z,U,p)
        
        # evaluate objective at current z
        #println("Evaluating objective")
        val = evaluate_objective(z,U,p)
        push!(values,val)
        
        # evaluate gradient at current z
        #println("Evaluating Gradient")
        evaluate_gradient!(z,U,grad,p)
           
        if(i%100 == 0)
            println("Iteration $i")
            println("current obj is - $val")
        end 
        
        #println("Performing Backtracking Line Search")
        # backtracking LS        
        t = 10
        # t = 1e4/(max_iter)
        z = z - t*grad
        
        solve_state!(z,U,p)
        obj = evaluate_objective(z,U,p)
        
        s = p.dt*ones(p.nT+1)
        s[1] /= 2
        s[p.nT+1] /= 2
        S = inv(Diagonal(s))
        
        iter = 1
        max_b = 10
        rhs = val - α*t*dot(grad,grad*S)
        
        
        while( obj > rhs )
            if(iter > max_b)
                break
            end 
            t = β*t
            z = z - t*grad
            solve_state!(z,U,p)
            obj = evaluate_objective(z,U,p)
            rhs = val - α*t*dot(grad,grad*S)
            iter += 1
        end
        #println("End of Iteration $i")
    end
    
    solve_state!(z,U,p)
    val = evaluate_objective(z,U,p)
    return z, U, val, values
end   

minimize (generic function with 1 method)

In [ ]:
p1 = parameters(250,500)
z01 = zeros(p1.nZ,p1.nT+1)*1e-4


z1,U1,val1,values1 = minimize(z01,p1)

Iteration 100
current obj is - -0.10839979103000318
Iteration 200
current obj is - -0.10865421453260851


In [33]:
p2 = parameters(1000,1200)
z02 = zeros(p2.nZ,p2.nT+1)*1e-4


z2,U2,val2,values2 = minimize(z02,p2)

LoadError: [91mUndefVarError: nT not defined[39m

In [25]:
values1

2-element Array{Any,1}:
 -0.155227
 -0.166689

In [ ]:
norms1[500]

In [ ]:
using Plots,PyPlot

In [ ]:
x = linspace(0,1,p1.nX); y = hcat(U1[:,150],p1.w[:,150])
pl = PyPlot.plot(x,y)

In [ ]:
size(values1)

In [ ]:
x = collect(1:1000)
y = values1-ones(1000)
PyPlot.plot(x,y)

In [ ]:
t = linspace(0,1,p1.nT); y = z1'
PyPlot.plot(t,y)

In [34]:
using JLD
D1 = values1;
save("Burger-values1.jld", "data", D1)
save("Burger-state1.jld", "data", U1)
save("Burger-control1.jld", "data", z1)


In [ ]:
p2 = parameters(1000,1200)
z02 = zeros(2,p2.nT)*1e-4


z2,U2,val2,values2 = minimize(z02,p2)

In [35]:
D2 = values2;
save("Burger-values2.jld", "data", D2)
save("Burger-state2.jld", "data", U2)
save("Burger-control2.jld", "data", z2)